$$\Huge \text{RAPORT}$$
$$\huge \text{LISTA 3}$$
$$\large \text{Autorzy: Katarzyna Rudzińska, Jagoda Płócienniczak}$$

## CZĘŚĆ I i II

### ZADANIE 1
    Napisz funkcję, która zwraca p-wartość w omówionym na wykładzie warunkowym teście symetrii w przypadku tabeli 2×2.

### ZADANIE 2
    W tabeli 1 umieszczono dane dotyczące reakcji na lek po godzinie od jego przyjęcia dla dwóch różnych leków przeciwbólowych stosowanych w migrenie. Leki zostały zaaplikowane grupie pacjentów w dwóch różnych atakach bólowych. Na podstawie danych zweryfikuj hipotezę, że leki te są jednakowo skuteczne korzystając z testu:

        a) McNemara z poprawką na ciągłość,
        b) warunkowego (korzystając z funkcji zadeklarowanej w zadaniu 1.) 

<div align="center">

<table>
  <thead>
    <tr>
      <th rowspan="2">Reakcja na lek A</th>
      <th colspan="2">Reakcja na lek B</th>
    </tr>
    <tr>
      <th>Negatywna</th>
      <th>Pozytywna</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Negatywna</strong></td>
      <td>1</td>
      <td>5</td>
    </tr>
    <tr>
      <td><strong>Pozytywna</strong></td>
      <td>2</td>
      <td>4</td>
    </tr>
  </tbody>
</table>

<br>
Table 1: Dane do zadania 2.


### ZADANIE 3
  Przeprowadź symulacje w celu porównania mocy testu Z i testu Z0 przedstawionych na wykładzie. Rozważ różne długości prób.

### ZADANIE 4
    Dla danych dołączonych do pierwszej listy zadań, na podstawie zmiennych CZY_ZADW oraz CZY_ZADW_2, zweryfikuj hipotezę, że zadowolenie ze szkoleń w pierwszym badanym okresie i w drugim badanym okresie pierwszego badania odpowiada modelowi symetrii. Czy na podstawie uzyskanych wników możemy wnioskować, że poziom zadowolenia ze szkoleń nie uległ zmianie? Przyjmij poziom istotności 0.05.

### ZADANIE 5
    W firmie, o której mowa w zadaniu 1 z listy 1, wdrożono pewne działania w celu poprawy komfortu pracy. Następnie badaną grupę respondentów ponownie poproszono o odpowiedź na pytanie dotyczące oceny podejścia firmy do umożliwiania wdrażania wiedzy zdobytej na szkoleniach. W Tabeli 2 przedstawiono tablicę dwudzielczą uwzględniającą odpowiedzi na pytanie w obu tych okresach. Na podstawie danych zweryfikuj hipotezę, że odpowiedzi w pierwszym badanym okresie i w drugim okresie odpowiadają modelowi symetrii. Na podstawie wyników uzyskanych przy weryfikacji hipotezy dotyczącej symetrii, sformułuj wniosek dotyczący hipotezy, że ocena podejścia firmy nie uległa zmianie.

<div align="center">

<table>
  <thead>
    <tr>
      <th rowspan="2">Pytanie 1</th>
      <th colspan="5">Pytanie 2</th>
    </tr>
    <tr>
      <th>-2</th>
      <th>-1</th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>-2</strong></td>
      <td>10</td>
      <td>2</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td><strong>-1</strong></td>
      <td>0</td>
      <td>15</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td><strong>0</strong></td>
      <td>1</td>
      <td>1</td>
      <td>32</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <td><strong>1</strong></td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>96</td>
      <td>3</td>
    </tr>
    <tr>
      <td><strong>2</strong></td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>26</td>
    </tr>
  </tbody>
</table>

<br>
Table 1: Dane do zadania 5.

## CZĘŚĆ III

### ZADANIE 6
    W pewnym badaniu porównywano skuteczność dwóch metod leczenia: Leczenie A to nowa procedura, a Leczenie B to stara procedura. Przeanalizuj dane przedstawione w Tabeli 3 (wyniki dla całej grupy pacjentów) oraz w Tabelach 4 i 5 (wyniki w podgrupach ze względu na dodatkową zmienną) i odpowiedz na pytanie, czy dla danych występuje paradoks Simpsona.

<div align="center">

<table>
  <thead>
    <tr>
      <th rowspan="2">Metoda</th>
      <th colspan="2">Wynik leczenia</th>
    </tr>
    <tr>
      <th>Poprawa</th>
      <th>Brak</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Leczenie A</strong></td>
      <td>117</td>
      <td>104</td>
    </tr>
    <tr>
      <td><strong>Leczenie B</strong></td>
      <td>177</td>
      <td>44</td>
    </tr>
  </tbody>
</table>
<br>
Table 3: Dane dla całej grupy

$\quad$

<table>
  <thead>
    <tr>
      <th rowspan="2">Metoda</th>
      <th colspan="2">Wynik leczenia</th>
    </tr>
    <tr>
      <th>Poprawa</th>
      <th>Brak</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Leczenie A</strong></td>
      <td>17</td>
      <td>101</td>
    </tr>
    <tr>
      <td><strong>Leczenie B</strong></td>
      <td>2</td>
      <td>36</td>
    </tr>
  </tbody>
</table>
<br>
Table 4: Dane dla pacjentów z chorobami współistniejącymi


$\quad$
<table>
  <thead>
    <tr>
      <th rowspan="2">Metoda</th>
      <th colspan="2">Wynik leczenia</th>
    </tr>
    <tr>
      <th>Poprawa</th>
      <th>Brak</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Leczenie A</strong></td>
      <td>100</td>
      <td>3</td>
    </tr>
    <tr>
      <td><strong>Leczenie B</strong></td>
      <td>175</td>
      <td>8</td>
    </tr>
  </tbody>
</table>

<br>
Table 5: Dane dla pacjentów bez chorób współistniejących

### ZADANIE 7
    Dla danych z listy 1, przyjmując za zmienną 1 zmienną CZY_KIER, za zmienną 2– zmienną PYT_2 i za zmienną 3– zmienną STAŻ, podaj interpretacje nastepujacych modeli log-liniowych: [1 3], [13], [1 2 3], [12 3], [12 13] oraz [1 23].

## CZĘŚĆ IV i V

### ZADANIE 8
    Przyjmując model log-liniowy [123] dla zmiennych opisanych w zadaniu 7 oszacuj prawdopobiebieństwa:
        a) że osoba pracująca na stanowisku kierowniczym jest zdecydowanie zadowolona ze szkoleń;
        b) że osoba o stażu pracy krótszym niż rok pracuje na stanowisku kierowniczym;
        c) że osoba o stażu pracy powyżej trzech lat nie pracuje na stanowisku kierowniczym.
    Jakie byłyby oszacowania powyższych prawdopodobieństw przy założeniu modelu [12 23]?

### ZADANIE 9
    Dla danych wskazanych w zadaniu 7 zweryfikuj następujące hipotezy:
        a) zmienne losowe CZY_KIER, PYT_2 i STAŻ są wzajemnie niezależne;
        b) zmienna losowa PYT_2 jest niezależna od pary zmiennych CZY_KIER i STAŻ;
        c)  zmienna losowa PYT_2 jest niezależna od zmiennej CZY_KIER, przy ustalonej wartości zmiennej STAŻ.

--------------------------------------------------

## ZADANIA DODATKOWE

### ZADANIE *1
    W przypadku zadania 5 występuje problem z zastosowaniem testu Bowkera ze względu na występowanie zer na określonych miejscach w tabeli z danymi. Zastosuj w tym przypadku dokładny test symetrii i opisz w jaki sposób wyznaczana jest wartość poziomu krytycznego w tym teście.

### ZADANIE *2
    Na podstawie danych z listy 1 dokonaj wyboru modelu rozważając uwzględnienie zmiennych PYT_1, PYT_2 i PŁEĆ w oparciu o:
        a) testy
        b) kryterium AIC
        c) kryterium BIC